In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset # Network Common Data Form

In [2]:
def VBOTAlignDatasets(time, lat, lon, data, time_obs, lat_obs, lon_obs, data_obs, userInputs):
    # For this function to operate properly, the time stamps on the hindcast and obs data need to be in 
    ## the form of "days since 2009-09-01"
    
    dayOfTheSeason = userInputs[0]
    lats           = userInputs[1]
    latn           = userInputs[2]
    lonw           = userInputs[3]
    lone           = userInputs[4]

    # Determine the index in the requested forecast for the desired day of the season 
    id_DayOfTheSeason = np.abs(time-dayOfTheSeason).argmin()

    # Finds the index of the value (in the observation dataset) that is equal to the first element of the hindcast
    id_time_begin = np.abs(time_obs-time[0]).argmin()
    id_time_end   = np.abs(time_obs-time[-1]).argmin()

    # NOTE TO SELF: this variable is currently unused...does Python de-allocate the memory upon exiting the function?
    # The plus one ensures that the final value is included
    time_obsRefined = time_obs[id_time_begin:id_time_end+1] 
    
    # Determine the indexes for region of interest
    idn = np.abs(lat-latn).argmin() # north
    ids = np.abs(lat-lats).argmin() # south
    
    # Align the longitude data depending on format
    if max(lon) > 190: # if data ranges from 0 - 360
        idw = np.abs(lon - (360+lonw)).argmin()
        ide = np.abs(lon - (360+lone)).argmin()
    else:              # if data ranges from -180 - 180
        idw = np.abs(lon-lonw).argmin()
        ide = np.abs(lon-lone).argmin()

    # Determine indexes for region of interest
    idn_obs = np.abs(lat_obs-latn).argmin()
    ids_obs = np.abs(lat_obs-lats).argmin()
    idw_obs = np.abs(lon_obs-(360+lonw)).argmin()
    ide_obs = np.abs(lon_obs-(360+lone)).argmin()
    
    # Obtain lat and lon values within the region of interest
    lat_plt     = lat[ids:idn]
    lon_plt     = lon[idw:ide] 
    lat_plt_obs = lat_obs[ids_obs:idn_obs] # Note that the latitude order ranges from south to north
    lon_plt_obs = lon_obs[idw_obs:ide_obs]
    
    # Select the data that aligns the observation data with the hindcast data on a temporal scale
    data_obsRefined = data_obs[id_time_begin:id_time_end+1][:]
    
    # Select the data in the user specified region of interest
    data_aligned     = data[:,ids:idn,idw:ide]
    data_aligned_obs = data_obsRefined[:,ids_obs:idn_obs,idw_obs:ide_obs]  # Note that the latitude order ranges from south to north
    
    return data_aligned, data_aligned_obs, lat_plt, lat_plt_obs, id_DayOfTheSeason